# Set up the environment

In [ ]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import seaborn
import os
import pickle
import json
import scipy.io as sio
from scipy.stats import spearmanr

# Define functions to create a filename

In [ ]:
def getFileName(name, n_samples, model_name, layer_name):
    return name \
        + "_{}_".format(n_samples) \
        + "_{}_".format(model_name) \
        + "_{}".format(layer_name)  \
        + ".npy"   

In [ ]:
def getFileNameSave(name, model_name, trained_on, tested_on):
    return name \
        + "_{}_".format(name) \
        + "_{}_".format(model_name) \
        + "_{}_".format(trained_on) \
        + "_{}_".format(tested_on) \  
        + ".npy"   

# Select the 60 scenes that were used for EEG

In [ ]:
def select_eeg_scenes():
    
    ''' Find indices of the 60 EEG scenes as fed to the DNN during testing,
        in order to find them in the DNN Input RDMs'''
    
    with open('selected_scenes_in_order.json') as json_file:
        scenes_all = json.load(json_file)
    with open('places365_60scenes.json') as json_file:
        scenes_60 = json.load(json_file)

    scenes_all_ids  = np.zeros([len(scenes_all),1])
    scenes_all_ids[:] = np.nan

    for idx,scene in enumerate(scenes_all):
        scenes_all_ids[idx] = int(scene[14:22]) 

    index_60_scenes = np.zeros([60,1])
    index_60_scenes[:] = np.nan

    for idx,scene in enumerate(scenes_60):
        id_scene = int(scene[14:22])
        index_60_scenes[idx] = np.where(scenes_all_ids == id_scene)[0]
        
    return index_60_scenes


# Define a function to create the upper triangular of Input RDMs

In [ ]:
def get_upper_triangular(rdm):
    num_conditions = rdm.shape[0] #num samples
    return rdm[np.triu_indices(num_conditions,1)] #take all above the main diagonal (excluding it), returns flattened version

# Specify the model and layers for RSA

In [ ]:
def define_layer_and_model_names(model_name,trained_on,tested_on,layers_path):
    
    ''' Get the layer names of the model of interest
        in order to load the appropriate Input RDMs. '''
    
    if trained_on == 'imagenet':
        if tested_on == 'imagenet':
            NR_OF_SAMPLES = 10000
            path_data = 'Objects/'
        elif tested_on == 'places':
            NR_OF_SAMPLES = 10220
            path_data = 'Objects/Places365/'

    elif trained_on == 'places':
        if tested_on == 'imagenet':
            NR_OF_SAMPLES = 10000
            path_data = 'Scenes/ImageNet/'
        elif tested_on == 'places':
            NR_OF_SAMPLES = 10220
            path_data = 'Scenes/Places365/'
            
    #select the relevant layers
    json_file_layers=os.path.join(layers_path, model_name + '_selected_layers.json') 
    with open(json_file_layers, "r") as fd:
        selected_layers = json.load(fd)

    #get the name of the layers
    num_layers = len(selected_layers) 
    print('Model: ', model_name, ', Number of layers: ',num_layers, ', trained on: ', trained_on)
    model_begin = 0 #index of the first layer of the desired model
    layer_names = []
    for i in range(num_layers):
        layer_names.append(selected_layers[model_begin+i].get('layer'))   
        
    return layer_names, path_data

# RSA: DNN vs EEG

In [ ]:
def RSA_EEG_DNN(model_name,trained_on,tested_on,layers_path):
    
    ''' Perform RSA on EEG RDMs at each timepoint and DNN RDMs at each layer. '''
    
    #load EEG data
    rdm_eeg_dict = sio.loadmat('rsa_cross_task_subjects_5_35')
    Input_rdm_eeg = rdm_eeg_dict.get('rdm_rsa') #not this one though! 
    
    #0) Get layer names
    layer_names, path_dnn = \
        define_layer_and_model_names(model_name,trained_on,tested_on,layers_path)
    
    #preallocate the Model RDM
    model_rdm = np.zeros(len(layer_names),rdm_eeg.shape[1])
    model_rdm[:] = np.nan
    
    #for each layer:
    for layer_idx,layer in enumerate(layer_names):
        #1) load the layer and get its upper triangular
        RDM_dnn_path = os.path.join(ROOT_PATH + path_dnn + 'Input_RDM/', \
                              getFileName('Input_RDM_', NR_OF_SAMPLES, model_name, layer))
        Input_RDM_dnn = np.load(RDM_dnn_path)   
        
        #2) select the 60 scenes from the input RDMs (10 220 by 10 220 one) - do it for each layer
        idx_eeg_scenes = select_eeg_scenes()
        Input_RDM_dnn_x = Input_RDM_dnn[idx_eeg_scenes,:]
        Input_RDM_dnn_selected = Input_RDM_dnn_x[:,idx_eeg_scenes]
        ut_rdm_dnn = get_upper_triangular(Input_RDM_dnn_selected)

        #3) for each timepoint: do RSA
        for timepoint in range(rdm_eeg.shape[0]): #not sure abt the dimension
            ut_rdm_eeg = get_upper_triangular(Input_rdm_eeg[0]) #check dimension
            
            print('Calculating the correlation distance between ->', layer_i, 'and', layer_j)
#             RSA = 1-spearmanr(ut_rdm_dnn,ut_rdm_eeg)[0]
            model_rdm[layer_idx,timepoint] = 1-spearmanr(ut_rdm_dnn,ut_rdm_eeg)[0]
#             RSA_matrix[np.where(np.array(layer_names_1)==layer_i)[0][0],\
#                    np.where(np.array(layer_names_2)==layer_j)[0][0]] = RSA_i_j

    return model_rdm

In [ ]:
def run_rsa_dnn_eeg(save_path,model_name,trained_on,tested_on):
    path = os.path.join(save_path, getFileNameSave('Model_RDM_EEG',model_name,trained_on,tested_on))
    RSA_EEG_DNN(model_name,trained_on,tested_on,'/mnt/raid/ni/agnessa/RSA/layer_names')
    print('Finished computing RSA between EEG data and ', model_name, ' trained on ', \ 
          trained_on, ' and tested on ', tested_on)

In [ ]:
run_rsa_dnn_eeg('/mnt/raid/ni/agnessa/RSA/RSA_EEG/Model_RDMs','alexnet','imagenet','places')
